In [2]:
%reload_ext autoreload
%reload_ext dotenv
%autoreload 2

import pydicom
import pynetdicom
from pynetdicom import AE, evt, build_role, debug_logger, sop_class
from pydicom import uid
import funcs
import dotenv
import pandas as pd
import os
import sys
sys.path.append('.')

%reload_ext funcs
%dotenv config.env

cannot find .env file


In [23]:
config = dotenv.dotenv_values() 
config_df = pd.DataFrame.from_dict(config ,  orient='index', columns=['value'])
config['PORT']

'11112'

In [7]:
import subprocess

os.makedirs( '/Users/personal-macbook/Documents/projects/D7.PACS/code/data/007' , exist_ok=True )

command = 'nohup python -m pynetdicom getscu --output-directory /Users/personal-macbook/Documents/projects/D7.PACS/code/data/007 www.dicomserver.co.uk 11112 -k QueryRetrieveLevel=PATIENT -k PatientID=007 | tee -a /Users/personal-macbook/Documents/projects/D7.PACS/code/data/007/007.log &'
p = subprocess.Popen('exec ' + command, stdout=subprocess.PIPE, shell=True)
out, err = p.communicate()

I: Requesting Association
I: Association Accepted
I: Sending Get Request: MsgID 1
I: 
I: # Request Identifier
I: (0008,0052) CS [PATIENT]                                # 1 QueryRetrieveLevel
I: (0010,0020) LO [007]                                    # 1 PatientID
I: 
I: Get SCP Result: 0x0000 (Success)
I: Sub-Operations Remaining: 0, Completed: 0, Failed: 0, Warning: 0
I: Releasing Association


In [10]:
import tqdm
import time 

tqdm.pause(10)

AttributeError: module 'tqdm' has no attribute 'pause'

## Send a C-echo request to a verification SCP

Verification SOP Class has a UID of:  1.2.840.10008.1.1

we can use the UID str directly when adding the requested presentation context

# <span style='color:green'> Supported DIMSE Services </span>
## <span style='color:orange'> SCU Services </span>
When the AE is acting as an SCU and an association has been established with a peer SCP, the following DIMSE-C and -N services are available:


| DIMSE service	| Association method |
|---	|---|
| C-ECHO	| Association.send_c_echo() |
| C-FIND	| Association.send_c_find(dataset, query_model) |
| C-GET	| Association.send_c_get(dataset, query_model) |
| C-MOVE	| Association.send_c_move(dataset, move_aet, query_model) |
| C-STORE	| Association.send_c_store(dataset) |
| N-ACTION	| Association.send_n_action(dataset, action_type, class_uid, instance_uid) |
| N-CREATE	| Association.send_n_create(dataset, class_uid, instance_uid) |
| N-DELETE	| Association.send_n_delete(class_uid, instance_uid) |
| N-EVENT-REPORT	| Association.send_n_event_report(dataset, event_type, class_uid, instance_uid) |
| N-GET	| Association.send_n_get(identifier_list, class_uid, instance_uid) |
| N-SET	| Association.send_n_set(dataset, class_uid, instance_uid) |

In [ ]:
addr = "www.dicomserver.co.uk"
port = 104 # 11112
ae_title="AET"
uid_verification_sop_class = '1.2.840.10008.1.1'


# Associate with a peer AE at IP
ae = AE(ae_title=ae_title)

ae.add_requested_context(uid_verification_sop_class)

associate = ae.associate(addr=addr, port=port)
print('associate is' + ('' if associate.is_established else 'not') + ' established')

# Example: Send a DIMSE C-ECHO request to the peer AE </span>
status = associate.send_c_echo()
print(status)

# associate.release()

# `send_c_find`

```python
AE().associate.send_c_get(dataset: Dataset, query_model: Union[str, UID], msg_id: int = 1, priority: int = 2)→ Iterator[Tuple[Dataset, Optional[Dataset]]][source]¶

```

Parameters
- `dataset` (`pydicom.dataset.Dataset`) – The `C-GET` request’s Identifier dataset. The exact requirements for the Identifier are Service Class specific (see the DICOM Standard, Part 4).

- `query_model` (`pydicom.uid.UID` or `str`) – The value to use for the `C-GET` request’s `(0000,0002)` Affected SOP Class UID parameter, which usually corresponds to the Information Model that is to be used.

- `msg_id` (`int`, `optional`) – The `C-GET` request’s Message ID, must be between `0` and `65535`, inclusive, (**`default 1`**).

- `priority` (`int`, `optional`) – The `C-GET` request’s Priority parameter, must be `0` (`Medium`), `1` (`High`) or `2` (`Low`) (**`default 2`**)




```bash
python -m pynetdicom findscu www.dicomserver.co.uk 104 -k QueryRetrieveLevel=STUDY

```

In [ ]:
from pynetdicom import AE, sop_class

connect_To_PACS = funcs.Connect_To_PACS()

connect_To_PACS.send_c_find(show_results=False, queryRetrieveLevel='STUDY', requestedContext=sop_class.PatientRootQueryRetrieveInformationModelFind)

In [ ]:
connect_To_PACS.list_sample_info[10][0]

# `send_c_get`

In [ ]:
connect_To_PACS = funcs.Connect_To_PACS(port=11112)

queryRetrieveLevel = 'PATIENT'
subject_info={ 'PatientID':'PAT009', 'StudyInstanceUID':None, 'SeriesInstanceUID':None}

connect_To_PACS.dataset(queryRetrieveLevel=queryRetrieveLevel, subject_info=subject_info)
connect_To_PACS.associate(requestedContext=sop_class.PatientStudyOnlyQueryRetrieveInformationModelGet)
connect_To_PACS.send_c_get(priority=0)

a = connect_To_PACS.assoc.send_c_get(dataset=connect_To_PACS.ds, query_model=connect_To_PACS.requestedContext, priority=2)

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/personal-macbook/Documents/test.csv')

for i in df.values:
    print(i)

In [ ]:
subject_info={ 'PatientID':'PAT009', 'StudyInstanceUID':None, 'SeriesInstanceUID':None}
a = subject_info.get('PatientID')
print(a)

In [ ]:
debug_logger()

# Implement the handler for evt.EVT_C_STORE
def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    ds.file_meta = event.file_meta

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID + '.dcm', write_like_original=False)

    # Return a 'Success' status
    return 0x0000

In [ ]:
# Initialise the Application Entity
ae = AE()

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(sop_class.PatientRootQueryRetrieveInformationModelGet)

# context = sop_class.CTImageStorage
# how to request all existing files

# Add the requested presentation context (Storage SCP)
# ae.add_requested_context(context)

# Create an SCP/SCU Role Selection Negotiation item for CT Image Storage
# role = build_role(context, scp_role=True)

handlers = [(evt.EVT_C_STORE, handle_store)]

# Associate with peer AE at IP 127.0.0.1 and port 11112
addr = "www.dicomserver.co.uk"
port = 104 # 11112
ae_title="AET"

[source](https://pydicom.github.io/pynetdicom/stable/apps/getscu.html)

```python
python -m pynetdicom getscu --output-directory "." www.dicomserver.co.uk 11112 -k QueryRetrieveLevel=PATIENT -k PatientID=832040


python -m pynetdicom getscu --output-directory "." www.dicomserver.co.uk 11112 -k QueryRetrieveLevel=STUDY -k (0008,0050)=<AccessionNumber>
```

In [ ]:
# assoc = ae.associate(addr=addr, port=port, ext_neg=[role], evt_handlers=handlers)
assoc = ae.associate(addr=addr, port=port)

# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level
ds = pydicom.dataset.Dataset()
ds.QueryRetrieveLevel = 'PATIENT'

# Unique key for PATIENT level
ds.PatientID = '832040'

# Unique key for STUDY level
# ds.StudyInstanceUID = '1.2.3'

# Unique key for SERIES level
# ds.SeriesInstanceUID = '2'

pynetdicom.get
if assoc.is_established:
    # Use the C-GET service to send the identifier
    responses = assoc.send_c_get(ds, sop_class.PatientRootQueryRetrieveInformationModelGet)

    for (status, identifier) in responses:
        if status:
            print('C-GET query status: 0x{0:04x}'.format(status.Status))
        else:
            print('Connection timed out, was aborted or received invalid response')

    # # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

In [ ]:
# responses_list = list(responses)

# responses_list

In [ ]:
# responses_list.__len__()

# Other Stuff

In [ ]:
# from dataclasses import dataclass

# # from pynetdicom.sop_class import VerificationSOPClass
# from pynetdicom.sop_class import VerificationServiceClass, Verification


# @dataclass
# class Modality:
#     addr: str
#     port: int
#     ae_title: str

# class Association:
#     def __init__(self, modality, context):
#         self.modality = modality
#         self.context = context

#     def __enter__(self):
#         ae = AE()
#         ae.add_requested_context(self.context)
#         self._association = ae.associate(**vars(self.modality))
#         return self._association

#     def __exit__(self, *args):
#         self._association.release()
#         self._association = None
        
        
# modality = Modality('www.dicomserver.co.uk', 104, 'AET')


# if __name__ == '__main__':
#     modality = Modality('www.dicomserver.co.uk', 104, 'AET')

#     with Association(modality, Verification) as assoc:
#         resp = assoc.send_c_echo()
#         # assoc.req
#         print(f'Modality responded with status: {resp.Status}')
        
#         # request the list of studies
#         resp = assoc.send_c_find()
#         # for (status, dataset) in resp:
#         #     print(f'Modality responded with status: {status}')
#         #     if dataset:
#         #         print(dataset)


In [ ]:
import pandas as pd
import logging
import numpy as np

In [ ]:
dir = '/Users/personal-macbook/Documents/projects/D7.PACS/code/test.csv'
df = pd.read_csv(dir)
dct = df.PatientID.to_dict()
dct
for name in dct:
   print(name)

In [ ]:
from app import Streamlit_App

streamlit_app = Streamlit_App()
# streamlit_app.getQueryRetrieveLevel()
# streamlit_app.start_download()
# streamlit_app.update_log()

log_dir = '/Users/personal-macbook/Documents/projects/D7.PACS/code/downloaded_images/007/007.log'

In [ ]:
with open(log_dir , 'r') as file:
    log_file = file.read().splitlines()

# Getting the indices for each downloaded file
last_line_index = [i for i, s in enumerate(log_file) if 'Releasing Association' in s]
index = [i for i, s in enumerate(log_file[:last_line_index[-1]]) if 'Storing DICOM file:' in s]

In [ ]:
from tqdm import tqdm

In [ ]:
df = pd.DataFrame()
dct = {}
for ix in tqdm(index):
    
    dct['case_name'] = log_file[ix].split('Storing DICOM file: ')[1]
    
    if 'W: DICOM file already exists, overwriting' in log_file[ix+1]:
        ix += 1
        
    dct['SCP_Response']  = log_file[ix+1].split('SCP Response:')[1]    
    dct[ 'Remaining'] = log_file[ix+2].split('Remaining:')[1].split(',')[0] ,
    dct['Completed']  = log_file[ix+2].split('Completed:')[1].split(',')[0] ,
    dct['Failed']         = log_file[ix+2].split('Failed:')[1].split(',')[0]
    
dct